In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import time

In [ ]:
model = tf.keras.models.load_model('tesla_road_model.keras')
IMG_SIZE = 256

@tf.function
def predict_frame(tensor):
    return model(tensor, training=False)

In [ ]:
cap = cv2.VideoCapture(0)

print("🚀 Tesla Vision ishga tushdi...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb, (IMG_SIZE, IMG_SIZE)) / 255.0
    input_tensor = tf.convert_to_tensor(np.expand_dims(resized.astype(np.float32), axis=0))

    start = time.perf_counter()
    pred = predict_frame(input_tensor)
    inference_time = time.perf_counter() - start

    mask = (pred.numpy()[0] > 0.5).astype(np.uint8)
    mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))

    overlay = frame.copy()
    overlay[mask_resized == 1] = [0, 255, 128]
    final_frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)

    fps_text = f"FPS: {1/inference_time:.1f} | Latency: {inference_time*1000:.1f}ms"
    cv2.putText(final_frame, fps_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Tesla Road Detection (M4 Power)', final_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()